In [1]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import os

In [2]:
url_base = "https://www.gasolinamx.com/gas/precio-gas-"
estados = ['aguascalientes', 'baja-california-norte','baja-california-sur','campeche','chiapas','chihuahua','cdmx','coahuila-de-zaragoza',
           'colima','durango','estado-de-mexico','guanajuato','guerrero', 'hidalgo','jalisco','michoacan','morelos','nayarit',
           'nuevo-leon','oaxaca', 'puebla','queretaro','quintana-roo','san-luis-potosi-slp','sinaloa','sonora','tabasco','tamaulipas',
           'tlaxcala','veracruz','yucatan','zacatecas']

estados_expresion = ['Aguascalientes', 'Baja California Norte','Baja California Sur','Campeche','Chiapas','Chihuahua','Ciudad de México - CDMX',
                    'Coahuila','Colima','Durango','Estado de México - EDOMEX','Guanajuato','Guerrero','Hidalgo','Jalisco',
                    'Michoacán','Morelos','Nayarit','Nuevo León','Oaxaca','Puebla','Querétaro','Quintana Roo','San Luis Potosí - SLP',
                    'Sinaloa','Sonora','Tabasco','Tamaulipas','Tlaxcala','Veracruz','Yucatán','Zacatecas']

data = []
fecha = datetime.datetime.now().strftime("%d-%m-%Y")

In [3]:
def obtener_contenido_pagina(url):
    pagina = requests.get(url)
    return pagina.content

In [4]:
def crear_soup(html):
    return BeautifulSoup(html, "html.parser")

In [5]:
def procesar_pagina(soup, estado, fecha):

    litros = []
    kilos = []
    tanques20kg = []
    
    patron_estados =  "|".join(re.escape(estado) for estado in estados_expresion)
    patron_precio = r"\$(\d+\.\d+)"

    litro =  r"El precio por litro de gas en ({}) es de {}".format(patron_estados, patron_precio)
    concuerdalitro = re.search(litro, soup.get_text())
    preciolitro = concuerdalitro.group(2)
    litros.append(float(preciolitro))


    kilo = r"El precio por kilo de gas en ({}) es de {}".format(patron_estados, patron_precio)
    concuerdakilo = re.search(kilo, soup.get_text())
    preciokilo = concuerdakilo.group(2)
    kilos.append(float(preciokilo))

    tanque20kg = r"El tanque de 20kg vale \$(\d+\.\d+)"
    concuerdatanque20gk = re.search(tanque20kg, soup.get_text())
    preciotanque20kg = concuerdatanque20gk.group(1)
    tanques20kg.append(float(preciotanque20kg))

    data.append({
            "Estado": estado,
            "Fecha": fecha,
            "precio_litros": litros[0],
            "precio_kilos": kilos[0],
            "precio_tanque20kg": tanques20kg[0]
        }) 

In [6]:
def manejar_paginacion(url_base):
   #fecha = datetime.datetime.now().strftime("%d-%m-%Y")
   for estado in estados:
      url = url_base + estado
      contenido_pagina = obtener_contenido_pagina(url)
      soup = crear_soup(contenido_pagina)
      procesar_pagina(soup, estado, fecha)
   dataframe = pd.DataFrame(data)
   return dataframe

In [7]:
df_hoy = manejar_paginacion(url_base)
df_hoy # Tiene 32 entradas

,Estado,Fecha,precio_litros,precio_kilos,precio_tanque20kg
0,aguascalientes,27-07-2023,8.68,16.07,321.4
1,baja-california-norte,27-07-2023,8.33,15.42,308.4
2,baja-california-sur,27-07-2023,10.77,19.95,399.0
3,campeche,27-07-2023,8.28,15.34,306.8
4,chiapas,27-07-2023,8.23,15.25,305.0
5,chihuahua,27-07-2023,8.47,15.69,313.8
6,cdmx,27-07-2023,8.10,15.00,300.0
7,coahuila-de-zaragoza,27-07-2023,8.49,15.73,314.6
8,colima,27-07-2023,8.43,15.62,312.4
9,durango,27-07-2023,9.23,17.09,341.8


In [8]:
def actualiza_datos(df_hoy):
    df_base = pd.read_csv("../csv/base.csv")
    df_actual = pd.concat([df_base, df_hoy], axis = 0)
    df_actual.to_csv("../csv/" + fecha + " actualizacion" + ".csv", index=False)
    return df_actual

In [9]:
actualizado = actualiza_datos(df_hoy)
actualizado

,Estado,Fecha,precio_litros,precio_kilos,precio_tanque20kg
0,aguascalientes,26-07-2023,8.68,16.07,321.4
1,baja-california-norte,26-07-2023,8.33,15.42,308.4
2,baja-california-sur,26-07-2023,10.77,19.95,399.0
3,campeche,26-07-2023,8.28,15.34,306.8
4,chiapas,26-07-2023,8.23,15.25,305.0
...,...,...,...,...,...
27,tamaulipas,27-07-2023,8.48,15.71,314.2
28,tlaxcala,27-07-2023,8.23,15.24,304.8
29,veracruz,27-07-2023,8.13,15.06,301.2
30,yucatan,27-07-2023,8.64,16.01,320.2
